In [1]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date # to_date pode ser útil para futuras transformações de data

# --- 1. Configuração do Ambiente e Caminhos ---
# A SparkSession já é gerenciada e disponibilizada automaticamente no Fabric.
# spark = SparkSession.builder.getOrCreate()

# Nomes do Lakehouse e do Schema de destino
# Ambos têm o mesmo nome.
lakehouse_name = "Projeto_II_Bronze_"
target_schema_name = "Projeto_II_Bronze_"

# Lista dos arquivos CSV que foram carregados manualmente para o diretório 'Files/'
csv_files_to_process = ["cities.csv", "product.csv", "sales.csv"]

# Caminho base para os arquivos CSV dentro do diretório 'Files/'
csv_source_path_in_lakehouse = "Files/"

print(f"Iniciando o processo da Camada Bronze para o Lakehouse: '{lakehouse_name}'")
print(f"Schema de destino para as tabelas Delta: '{target_schema_name}'")
print(f"Arquivos CSV a serem processados: {csv_files_to_process}")
print(f"Caminho de origem dos CSVs (dentro do Lakehouse): '{csv_source_path_in_lakehouse}'")
print("-" * 80)

# --- 2. Função Auxiliar para Processar um CSV e Salvar como Delta Table ---

def process_csv_to_delta_bronze(spark_session, file_name, source_base_path, lh_name, schema_name):
    """
    Lê um arquivo CSV de um caminho especificado no Lakehouse 'Files/'
    e o salva como uma tabela Delta em um schema específico dentro do mesmo Lakehouse.
    """
    full_csv_path = f"{source_base_path}{file_name}"
    
    # O nome da tabela Delta será o nome do arquivo CSV (sem a extensão) + sufixo "_bronze"
    # Ex: cities.csv -> cities_bronze
    table_base_name = file_name.replace(".csv", "")
    full_delta_table_name = f"{schema_name}.{table_base_name}_bronze" # Nome qualificado com o schema

    print(f"\nProcessando '{file_name}'...")
    print(f"Caminho completo do CSV de origem: {full_csv_path}")
    print(f"Nome da Tabela Delta de destino: {lh_name}.{full_delta_table_name}")

    try:
        # Lendo o arquivo CSV para um DataFrame Spark
        # .option("header", "true") -> Assume que a primeira linha contém os nomes das colunas
        # .option("inferSchema", "true") -> Tenta automaticamente adivinhar os tipos de dados das colunas
        df_raw = spark_session.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .csv(full_csv_path)

        print(f"DataFrame para '{file_name}' lido com sucesso. Exemplo de Schema Inferido:")
        df_raw.printSchema()
        print(f"Número de linhas detectadas: {df_raw.count()}")

        # Salvando o DataFrame como uma tabela Delta no schema especificado do Lakehouse.
        # .format("delta") -> Especifica o formato Delta Lake.
        # .mode("overwrite") -> Se a tabela já existir, ela será completamente substituída.
        # .saveAsTable(f"{lh_name}.{full_delta_table_name}") -> Salva o DataFrame como uma tabela gerenciada
        #                                                     no Catálogo do Spark e no Lakehouse,
        #                                                     no schema especificado.
        df_raw.write \
            .format("delta") \
            .mode("overwrite") \
            .saveAsTable(f"{lh_name}.{full_delta_table_name}")

        print(f"Tabela Delta '{full_delta_table_name}' criada/atualizada com sucesso no Lakehouse '{lh_name}'.")

    except Exception as e:
        print(f"ERRO: Falha ao processar '{file_name}'. Detalhes do erro: {e}")

# --- 3. Executar o Processamento para cada Arquivo CSV ---

for csv_file in csv_files_to_process:
    process_csv_to_delta_bronze(spark, csv_file, csv_source_path_in_lakehouse, lakehouse_name, target_schema_name)

# --- 4. Verificação Final das Tabelas Criadas (Opcional) ---
print("\n" + "="*80)
print("--- VERIFICAÇÃO: Tabelas Delta Criadas na Camada Bronze ---")
print("="*80)

try:
    print(f"\nListando tabelas no schema '{target_schema_name}' do Lakehouse '{lakehouse_name}':")
    # Usa Spark SQL para listar as tabelas no schema especificado
    spark.sql(f"SHOW TABLES IN {lakehouse_name}.{target_schema_name}").show()

    # Mostrar algumas linhas de uma das tabelas para confirmar
    print(f"\nExemplo de dados da tabela '{target_schema_name}.sales_bronze':")
    # Lendo a tabela Delta recém-criada
    df_sales_bronze_check = spark.read.format("delta").table(f"{lakehouse_name}.{target_schema_name}.sales_bronze")
    df_sales_bronze_check.show(5) # Mostra as 5 primeiras linhas
    df_sales_bronze_check.printSchema() # Mostra o schema da tabela Delta

except Exception as e:
    print(f"Erro durante a verificação final: {e}")

print("\n--- Processamento da Camada Bronze Concluído com Sucesso! ---")

# spark.stop() # Geralmente não é necessário no Fabric, a sessão é gerenciada automaticamente

StatementMeta(, e6e9fd47-233d-422e-aac2-962fa76a3b4b, 3, Finished, Available, Finished)

Iniciando o processo da Camada Bronze para o Lakehouse: 'Projeto_II_Bronze_'
Schema de destino para as tabelas Delta: 'Projeto_II_Bronze_'
Arquivos CSV a serem processados: ['cities.csv', 'product.csv', 'sales.csv']
Caminho de origem dos CSVs (dentro do Lakehouse): 'Files/'
--------------------------------------------------------------------------------

Processando 'cities.csv'...
Caminho completo do CSV de origem: Files/cities.csv
Nome da Tabela Delta de destino: Projeto_II_Bronze_.Projeto_II_Bronze_.cities_bronze
DataFrame para 'cities.csv' lido com sucesso. Exemplo de Schema Inferido:
root
 |-- store_id: string (nullable = true)
 |-- storetype_id: string (nullable = true)
 |-- store_size: integer (nullable = true)
 |-- city_id_old: string (nullable = true)
 |-- country_id: string (nullable = true)
 |-- city_code: string (nullable = true)

Número de linhas detectadas: 63
Tabela Delta 'Projeto_II_Bronze_.cities_bronze' criada/atualizada com sucesso no Lakehouse 'Projeto_II_Bronze_'.
